In [1]:
import findspark
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from pyspark.sql import Row
import numpy as np
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
findspark.init()

In [2]:
#import income.csv as income
file = 'income.csv'
income = pd.read_csv(file)
income.head(50)

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
0,40,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,70,United-States,>50K
1,30,Private,HS-grad,9,Divorced,Craft-repair,Not-in-family,White,Female,40,United-States,<=50K
2,46,Private,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,United-States,>50K
3,32,Private,Assoc-voc,11,Married-civ-spouse,Other-service,Husband,White,Male,60,United-States,>50K
4,63,Private,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,16,United-States,<=50K
5,25,Private,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,40,Mexico,<=50K
6,71,Local-gov,HS-grad,9,Widowed,Exec-managerial,Other-relative,White,Female,33,United-States,<=50K
7,37,Private,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,40,United-States,<=50K
8,44,Private,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,Black,Male,40,United-States,<=50K
9,21,Private,HS-grad,9,Never-married,Protective-serv,Own-child,White,Male,40,United-States,<=50K


In [3]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23149 entries, 0 to 23148
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             23149 non-null  int64 
 1   workclass       23149 non-null  object
 2   education       23149 non-null  object
 3   education_num   23149 non-null  int64 
 4   marital_status  23149 non-null  object
 5   occupation      23149 non-null  object
 6   relationship    23149 non-null  object
 7   race            23149 non-null  object
 8   sex             23149 non-null  object
 9   hours_per_week  23149 non-null  int64 
 10  native_country  23149 non-null  object
 11  income          23149 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.1+ MB


In [4]:
income_cat = income.dtypes[income.dtypes == 'object'].index.tolist()

In [5]:
df = income

In [6]:
#check the number of unique values in each column
income[income_cat].nunique()

workclass          7
education         16
marital_status     7
occupation        14
relationship       6
race               5
sex                2
native_country    40
income             2
dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(income[income_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(income_cat)
encode_df.head()

d:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
income = income.merge(encode_df,left_index=True, right_index=True)
income = income.drop(income_cat,1)
income.head()

C:\Users\geoff\AppData\Local\Temp\ipykernel_24612\2929446180.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  income = income.drop(income_cat,1)


,age,education_num,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K
0,40,15,70,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,30,9,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,46,10,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,32,11,60,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,63,10,16,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [10]:
#drop capital.gain, capital.loss, fnlwgt, income_>50K
income.drop(['income_>50K'], axis=1, inplace=True)

# Split our preprocessed data into our features and target arrays
y = income["income_<=50K"].values
X = income.drop(["income_<=50K"], axis=1).values


In [11]:

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 808       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 859 (3.36 KB)
Trainable params: 859 (3.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


543/543 [==============================] - 1s 1ms/step - loss: 0.4441 - accuracy: 0.7670
Epoch 2/100
543/543 [==============================] - 0s 833us/step - loss: 0.3848 - accuracy: 0.8178
Epoch 3/100
543/543 [==============================] - 0s 857us/step - loss: 0.3709 - accuracy: 0.8267
Epoch 4/100
543/543 [==============================] - 0s 796us/step - loss: 0.3631 - accuracy: 0.8313
Epoch 5/100
543/543 [==============================] - 0s 757us/step - loss: 0.3579 - accuracy: 0.8339
Epoch 6/100
543/543 [==============================] - 0s 763us/step - loss: 0.3540 - accuracy: 0.8356
Epoch 7/100
543/543 [==============================] - 0s 860us/step - loss: 0.3513 - accuracy: 0.8366
Epoch 8/100
543/543 [==============================] - 0s 910us/step - loss: 0.3483 - accuracy: 0.8381
Epoch 9/100
543/543 [==============================] - 0s 769us/step - loss: 0.3469 - accuracy: 0.8368
Epoch 10/100
543/543 [==============================] - 0s 810us/step - loss: 0.3446 - 

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

181/181 - 0s - loss: 0.3676 - accuracy: 0.8355 - 221ms/epoch - 1ms/step
Loss: 0.36761921644210815, Accuracy: 0.8355217576026917
